# Curating a genome scale model

This notebook has been tested on [jprime.lbl.gov](jprime.lbl.gov) with the biodesign_3.7 kernel.

It starts with the model that gets output by the annotation_gr.ipynb notebook.

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from IPython.display import IFrame
import numpy as np
import pandas as pd
import json
import urllib
import cobra
import cplex
import os
import requests
import collections
import itertools

# Getting and preparing the genome-scale model

## Load *R.opacus* NCBI model generated by CarveMe

In [47]:
model = cobra.io.read_sbml_model("GSMs/Ropacus_annotated.xml")
model

Name,ropacus_annotated
Memory address,0x07f4a48afd410
Number of metabolites,1581
Number of reactions,2380
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


## Starting MEMOTE Output

In [74]:
IFrame('memotes/ropacus_annotated.htm', 1500, 800)

Define functions to print status report

In [49]:
def should_be_balanced(r):
    return not (r.id.startswith('EX_') or r.id.startswith('sink_') or r.id.startswith('Growth'))

def has_metabolite_with_multiple_formulas(r):
    return len([m for m in r.metabolites if len(m.formula.split(';')) > 1]) > 0
    
def status_report():
    for i in range(1,5):
        num_formulas = [m for m in model.metabolites if len(m.formula.split(';')) == i]
        print(f'{len(num_formulas)} of {len(model.metabolites)} metabolites have {i} formula(s)')
    print('\n')
    
    unbalanced = [r for r in model.reactions if should_be_balanced(r) and r.check_mass_balance() != {}]
    unbalanced_but_okay = [r for r in model.reactions if not should_be_balanced(r) and r.check_mass_balance() != {}]
    balanced = [r for r in model.reactions if r.check_mass_balance() == {}]
    
    unbalanced_multiple_formulas = [r for r in unbalanced if has_metabolite_with_multiple_formulas(r)]
    unbalanced_but_okay_multiple_formulas = [r for r in unbalanced_but_okay if has_metabolite_with_multiple_formulas(r)]
    balanced_multiple_formulas   = [r for r in   balanced if has_metabolite_with_multiple_formulas(r)]
    
    print(f'{len(unbalanced)} of the {len(model.reactions)} reactions in the model are wrongly unbalanced')
    print(f'{len(unbalanced_but_okay)} of the {len(model.reactions)} reactions in the model are properly unbalanced')
    print(f'{len(balanced)} of the {len(model.reactions)} reactions in the model are balanced')
    print('\n')
    
    print(f'{len(unbalanced_multiple_formulas)} of the {len(unbalanced)} improperly unbalanced reactions in the model have at least one metabolite with multiple formulas')
    print(f'{len(unbalanced_but_okay_multiple_formulas)} of the {len(unbalanced_but_okay)} properly unbalanced reactions in the model have at least one metabolite with multiple formulas')
    print(f'{len(balanced_multiple_formulas)} of the {len(balanced)} balanced reactions in the model have at least one metabolite with multiple formulas')

In [50]:
status_report()

1419 of 1581 metabolites have 1 formula(s)
156 of 1581 metabolites have 2 formula(s)
5 of 1581 metabolites have 3 formula(s)
1 of 1581 metabolites have 4 formula(s)


850 of the 2380 reactions in the model are wrongly unbalanced
228 of the 2380 reactions in the model are properly unbalanced
1302 of the 2380 reactions in the model are balanced


847 of the 850 improperly unbalanced reactions in the model have at least one metabolite with multiple formulas
19 of the 228 properly unbalanced reactions in the model have at least one metabolite with multiple formulas
27 of the 1302 balanced reactions in the model have at least one metabolite with multiple formulas


Inspect the 3 reactions that are imbalanced not due to undefined metabolites

In [51]:
unbalanced = [r for r in model.reactions if should_be_balanced(r) and r.check_mass_balance() != {}]

for r in [r for r in unbalanced if not has_metabolite_with_multiple_formulas(r)]:
    print(r.check_mass_balance())

{'C': -7.105427357601002e-15, 'H': -1.4210854715202004e-14, 'N': -8.881784197001252e-16, 'O': -3.552713678800501e-15, 'S': -2.220446049250313e-16}
{'C': -7.105427357601002e-15, 'O': -3.552713678800501e-15, 'N': -8.881784197001252e-16, 'S': -2.220446049250313e-16}
{'H': 2.6645352591003757e-15}


These have extremely small errors, and will be ignored. I'm not sure how to fix them

## Fix equivalent metabolite formulas
Define functions for this section

In [52]:
def get_initial_number_string(substring):
    initial_string = ''
    for char in substring:
        if char.isdigit():
            initial_string += char
        else:
            return initial_string
    return initial_string

def formula_dict_from_string(formula_string):
    formula_dict = {}
    elements = [char for char in formula_string if char.isalpha()]
    for element in elements:
        string_after_element = formula_string.split(element, 1)[1]
        coefficient = get_initial_number_string(string_after_element)
        if coefficient == '':
            coefficient = '1'
        formula_dict[element] = int(coefficient)
    return formula_dict

def all_formulas_equivalent(m):
    first_formula = m.formula.split(';')[0]
    return len([f for f in m.formula.split(';') if formula_dict_from_string(f) != formula_dict_from_string(first_formula)]) > 0

In [53]:
equivalent_formulas = 0
for m in [m for m in model.metabolites if ';' in m.formula and not all_formulas_equivalent(m)]:
    m.formula = m.formula.split(';')[0]
    equivalent_formulas += 1

print(f'There are {equivalent_formulas} metabolites with equivalent formulas, and they have been fixed.')

There are 26 metabolites with equivalent formulas, and they have been fixed.


In [54]:
status_report()

1445 of 1581 metabolites have 1 formula(s)
130 of 1581 metabolites have 2 formula(s)
5 of 1581 metabolites have 3 formula(s)
1 of 1581 metabolites have 4 formula(s)


333 of the 2380 reactions in the model are wrongly unbalanced
228 of the 2380 reactions in the model are properly unbalanced
1819 of the 2380 reactions in the model are balanced


330 of the 333 improperly unbalanced reactions in the model have at least one metabolite with multiple formulas
14 of the 228 properly unbalanced reactions in the model have at least one metabolite with multiple formulas
27 of the 1819 balanced reactions in the model have at least one metabolite with multiple formulas


# Assign metabolite formulas by checking if one formula makes all reactions the where its the only undefined metabolite balanced
Define functions for this section

In [55]:
def m_only_undefined_metabolite(m1, r):
    return ';' in m1.formula and len([m2 for m2 in r.metabolites if ';' in m2.formula and m1 != m2]) == 0

def reactions_where_m_is_only_undefined_metabolite(m):
    return [r for r in m.reactions if m_only_undefined_metabolite(m,r)]

def fraction_of_reactions_formula_balances(m, formula, rxn_list):
    original_formula = m.formula
    m.formula = formula
    balanced_reactions   = [r for r in rxn_list if r.check_mass_balance() == {}]
    unbalanced_reactions = [r for r in rxn_list if r.check_mass_balance() != {}]
    m.formula = original_formula
    
    # avoid divide by zero
    if len(balanced_reactions) + len(unbalanced_reactions) == 0:
        return 0
    return len(balanced_reactions) / (len(balanced_reactions) + len(unbalanced_reactions))

Run this function until no additional metabolites can be defined based on being the only undefined metabolite. Only assign formulas to perfect fits (Need to improve this wording)

In [56]:
metabolites_that_can_be_defined = 1
while metabolites_that_can_be_defined > 0:
    metabolites_that_can_be_defined = 0
    for m in [m for m in model.metabolites if ';' in m.formula]:
        for f in m.formula.split(';'):
            if fraction_of_reactions_formula_balances(m, f, reactions_where_m_is_only_undefined_metabolite(m)) == 1:
                metabolites_that_can_be_defined += 1
                m.formula = f

    print(f'{metabolites_that_can_be_defined} metabolites can be defined in this round')

66 metabolites can be defined in this round
19 metabolites can be defined in this round
14 metabolites can be defined in this round
4 metabolites can be defined in this round
1 metabolites can be defined in this round
0 metabolites can be defined in this round


In [57]:
status_report()

1549 of 1581 metabolites have 1 formula(s)
30 of 1581 metabolites have 2 formula(s)
2 of 1581 metabolites have 3 formula(s)
0 of 1581 metabolites have 4 formula(s)


91 of the 2380 reactions in the model are wrongly unbalanced
228 of the 2380 reactions in the model are properly unbalanced
2061 of the 2380 reactions in the model are balanced


88 of the 91 improperly unbalanced reactions in the model have at least one metabolite with multiple formulas
14 of the 228 properly unbalanced reactions in the model have at least one metabolite with multiple formulas
5 of the 2061 balanced reactions in the model have at least one metabolite with multiple formulas


# Now repeat but allow for imperfect fitting
Assign formulas that satisfy the greatest fraction of reactions first.
After each loop check to see which formulas have the greatest fraction of reactions they balance

In [58]:
highest_fraction = 1
while highest_fraction > 0:
    highest_fraction = 0
    metabolites_that_can_be_defined = 0

    # find highest fraction of reactions that are solved by a given formula
    for m in [m for m in model.metabolites if ';' in m.formula]:
        for f in m.formula.split(';'):
            if fraction_of_reactions_formula_balances(m, f, reactions_where_m_is_only_undefined_metabolite(m)) > highest_fraction:
                highest_fraction = fraction_of_reactions_formula_balances(m, f, reactions_where_m_is_only_undefined_metabolite(m))

    # assign formulas to metabolites with formula that gives a score equal to the best fraction
    if highest_fraction > 0:
        for m in [m for m in model.metabolites if ';' in m.formula]:
            for f in m.formula.split(';'):
                if fraction_of_reactions_formula_balances(m, f, reactions_where_m_is_only_undefined_metabolite(m)) == highest_fraction:
                    m.formula = f
                    metabolites_that_can_be_defined += 1

    print(f'{metabolites_that_can_be_defined} metabolites can be defined in this round with a fitting score of {highest_fraction}')

1 metabolites can be defined in this round with a fitting score of 0.8666666666666667
1 metabolites can be defined in this round with a fitting score of 1.0
1 metabolites can be defined in this round with a fitting score of 0.8571428571428571
4 metabolites can be defined in this round with a fitting score of 0.75
4 metabolites can be defined in this round with a fitting score of 0.6666666666666666
1 metabolites can be defined in this round with a fitting score of 0.6
2 metabolites can be defined in this round with a fitting score of 1.0
13 metabolites can be defined in this round with a fitting score of 0.5
0 metabolites can be defined in this round with a fitting score of 0


In [59]:
status_report()

1576 of 1581 metabolites have 1 formula(s)
4 of 1581 metabolites have 2 formula(s)
1 of 1581 metabolites have 3 formula(s)
0 of 1581 metabolites have 4 formula(s)


34 of the 2380 reactions in the model are wrongly unbalanced
228 of the 2380 reactions in the model are properly unbalanced
2118 of the 2380 reactions in the model are balanced


17 of the 34 improperly unbalanced reactions in the model have at least one metabolite with multiple formulas
0 of the 228 properly unbalanced reactions in the model have at least one metabolite with multiple formulas
0 of the 2118 balanced reactions in the model have at least one metabolite with multiple formulas


# Assign remaining metabolites by checking if combinations of formulas are balanaced or only off by hydrogen
Note 1: there is a preference for more verbose formulas to minimize shorthand notation<br>
Note 2: This is only done if there are less than 10 metabolites since the number of formula combination grows roughly exponentially (i.e. 10 undefined metabolites each with 2 formulas yields 2^10 possible combinations <br>
Define functions for this section

In [60]:
def is_balanced(r):
    return abs(sum(list(r.check_mass_balance().values()))) < 1e-5

def balanced_or_only_hydrogen_unbalanced(r):
    return should_be_balanced(r) and (is_balanced(r) or list(r.check_mass_balance().keys()) == ['H'])

def reactions_off_by_more_than_hydrogen():
    return [r for r in model.reactions if should_be_balanced(r) and not(balanced_or_only_hydrogen_unbalanced(r))]

def chars_in_string_list(string_list):
    total_chars = 0
    for string in string_list:
        total_chars += len(string)
    return total_chars


In [61]:
undefined_metabolites = [m for m in model.metabolites if ';' in m.formula]
possible_formulas = [m.formula.split(';') for m in model.metabolites if ';' in m.formula]

# only do this step if there is a reasonable number of undefined metbolites due to exponential growth of formula combinations
if len(undefined_metabolites) < 10:
    
    # inital best formulas is their original values
    best_formulas = [m.formula for m in undefined_metabolites]
    best_score = len(reactions_off_by_more_than_hydrogen())
    best_length = 0

    # goes through all permutations of formulas for undefined metabolites
    for formulas in list(itertools.product(*possible_formulas)):
        # assign the formulas to the metabolites
        for count, m in enumerate(undefined_metabolites):
            model.metabolites.get_by_id(m.id).formula = formulas[count]

        # get the number of reactions that are off by more than hydrogen
        unacceptable_reactions = reactions_off_by_more_than_hydrogen()
#         print(len(unacceptable_reactions), formulas) this line give a lot of details

        # if its the best fit replace the best formulas
        if len(unacceptable_reactions) <= best_score:
            if chars_in_string_list(formulas) > best_length:
                best_formulas = formulas
                best_score = len(reactions_off_by_more_than_hydrogen())
                best_length = chars_in_string_list(formulas)
                
for count, m in enumerate(undefined_metabolites):
    model.metabolites.get_by_id(m.id).formula = best_formulas[count]
    print(f'For metabolite {m.id}, with possible formulas {possible_formulas[count]}, {best_formulas[count]} was chosen')

For metabolite trdox_c, with possible formulas ['X', 'C6H7NO2S2R2'], C6H7NO2S2R2 was chosen
For metabolite trdrd_c, with possible formulas ['XH2', 'C6H9NO2S2R2'], C6H9NO2S2R2 was chosen
For metabolite fdxox_c, with possible formulas ['X', 'Fe8S8X'], Fe8S8X was chosen
For metabolite glutrna_c, with possible formulas ['C5H7NO3R', 'C5H8NO4X'], C5H8NO4X was chosen
For metabolite trnaglu_c, with possible formulas ['R', 'HOX', 'C15H21N5O10PR'], HOX was chosen


In [62]:
status_report()

1581 of 1581 metabolites have 1 formula(s)
0 of 1581 metabolites have 2 formula(s)
0 of 1581 metabolites have 3 formula(s)
0 of 1581 metabolites have 4 formula(s)


19 of the 2380 reactions in the model are wrongly unbalanced
228 of the 2380 reactions in the model are properly unbalanced
2133 of the 2380 reactions in the model are balanced


0 of the 19 improperly unbalanced reactions in the model have at least one metabolite with multiple formulas
0 of the 228 properly unbalanced reactions in the model have at least one metabolite with multiple formulas
0 of the 2133 balanced reactions in the model have at least one metabolite with multiple formulas


# Balance Hydrogen
Define function to balance hydrogen

In [63]:
def only_hydrogen_unbalanced(r):
    return list(r.check_mass_balance().keys()) == ['H'] and should_be_balanced(r)

def fix_unbalanced_hydrogen(r):
    hydrogen_error = int(r.check_mass_balance()['H'])
    r.subtract_metabolites({model.metabolites.get_by_id("h_c"): hydrogen_error})

In [64]:
for r in [r for r in model.reactions if only_hydrogen_unbalanced(r)]:
    fix_unbalanced_hydrogen(r)

In [65]:
status_report()

1581 of 1581 metabolites have 1 formula(s)
0 of 1581 metabolites have 2 formula(s)
0 of 1581 metabolites have 3 formula(s)
0 of 1581 metabolites have 4 formula(s)


5 of the 2380 reactions in the model are wrongly unbalanced
228 of the 2380 reactions in the model are properly unbalanced
2147 of the 2380 reactions in the model are balanced


0 of the 5 improperly unbalanced reactions in the model have at least one metabolite with multiple formulas
0 of the 228 properly unbalanced reactions in the model have at least one metabolite with multiple formulas
0 of the 2147 balanced reactions in the model have at least one metabolite with multiple formulas


# For now, remove reactions that are not balanced
Should probably change this in the future

In [66]:
for r in [r for r in model.reactions if should_be_balanced(r) and r.check_mass_balance() != {}]:
    print(r, '\n', r.check_mass_balance(), '\n')

AGPATr_BS: 0.01 1ag3p_BS_c + 0.07 fa11coa_c + 0.17 fa12coa_c + 0.01 fa1coa_c + 0.2 fa3coa_c + 0.34 fa4coa_c + 0.05 fa6coa_c + 0.1 pmtcoa_c + 0.03 strcoa_c + 0.03 tdcoa_c <=> 0.01 12dag3p_BS_c + coa_c 
 {'C': -7.105427357601002e-15, 'H': -1.4210854715202004e-14, 'N': -8.881784197001252e-16, 'O': -3.552713678800501e-15, 'S': -2.220446049250313e-16} 

FRDO: fdxrd_c + nadp_c <=> fdxox_c + h_c + nadph_c 
 {'H': 2.0, 'Fe': 6.0, 'S': 6.0} 

G3POA_BS: 0.07 fa11coa_c + 0.17 fa12coa_c + 0.01 fa1coa_c + 0.2 fa3coa_c + 0.34 fa4coa_c + 0.05 fa6coa_c + glyc3p_c + 0.1 pmtcoa_c + 0.03 strcoa_c + 0.03 tdcoa_c --> 0.01 1ag3p_BS_c + coa_c 
 {'C': -7.105427357601002e-15, 'O': -3.552713678800501e-15, 'N': -8.881784197001252e-16, 'S': -2.220446049250313e-16} 

NADH8: 2dmmq8_c + 3.8 h_c + nadh_c --> 2dmmql8_c + 2.8 h_e + nad_c 
 {'H': 2.6645352591003757e-15} 

OOR3r: akg_c + coa_c + fdxox_c + h_c --> co2_c + fdxrd_c + succoa_c 
 {'H': -2.0, 'S': -6.0, 'Fe': -6.0} 



In [67]:
for r in [r for r in model.reactions if should_be_balanced(r) and r.check_mass_balance() != {}]:
    r.remove_from_model(remove_orphans = True)

In [68]:
status_report()

1579 of 1579 metabolites have 1 formula(s)
0 of 1579 metabolites have 2 formula(s)
0 of 1579 metabolites have 3 formula(s)
0 of 1579 metabolites have 4 formula(s)


0 of the 2375 reactions in the model are wrongly unbalanced
228 of the 2375 reactions in the model are properly unbalanced
2147 of the 2375 reactions in the model are balanced


0 of the 0 improperly unbalanced reactions in the model have at least one metabolite with multiple formulas
0 of the 228 properly unbalanced reactions in the model have at least one metabolite with multiple formulas
0 of the 2147 balanced reactions in the model have at least one metabolite with multiple formulas


# Save the curated model

In [71]:
model.id = 'ropacus_curated_with_5_deleted_reactions'
model.name = 'Rhodococcus opacus PD630 with 5 reaction deletions'
model.description = 'Rhodococcus opacus PD630 model with 5 reaction deletions'

cobra.io.write_sbml_model(model, "GSMs/Ropacus_5_reaction_deletions.xml")

In [72]:
model

Name,ropacus_curated_with_5_deleted_reactions
Memory address,0x07f4a48afd410
Number of metabolites,1579
Number of reactions,2375
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


# Check memote of current model

In [75]:
IFrame('memotes/ropacus_curated_with_5_deleted_reactions.htm', 1500, 800)